# App Data
Our aim is to help developers understand which type of apps are likely to attract more users on Google Play and the App Store.

## Opening & Exploring Data
First we are going to open the 2 datasets

In [34]:
from csv import reader

opened_appstore = open('AppleStore.csv')
read_appstore = reader(opened_appstore)
list_appstore = list(read_appstore)
appstore_header = list_appstore[0]
appstore = list_appstore[1:]

opened_google = open('googleplaystore.csv')
read_google = reader(opened_google)
list_google = list(read_google)
google_header = list_google[0]
google = list_google[1:]

Created function to more easily explore certain sections of the datasets 

In [35]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [36]:
explore_data(list_appstore,0,1,True)
print('\n')
explore_data(list_google,0,1,True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


Number of rows: 7198
Number of columns: 16


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


Number of rows: 10842
Number of columns: 13


The apple store has 16 columns and 7197 apps

The google play store has 13 columns and 10841 apps

## Deleting Wrong Data
The Google Play dataset has a discussion section online, through reading on here we can see there is an error with row 10472

In [37]:
print(google[10472]) 
print('\n')
print(google_header)  
print('\n')
print(google[0]) 

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


The positions for the columns in 'Life Made WI-Fi Touchscreen Photo Frame' are off so we will delete this row

In [38]:
print(len(google))
del google[10472]
print(len(google))

10841
10840


## Removing Duplicate Entries
Some apps have multiple entries, here we will remove them and keep the most recent entry

In [39]:
for app in google:
    name = app[0]
    if name == "Instagram":
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


Above we show an example, the Instagram app shows up 4 times.

In [40]:
unique_apps = []
duplicate_apps = []

for app in google:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
        
print(len(duplicate_apps))

1181


Here we show that there are 1181 duplicate apps.

If you look at the 4 entries for Instagram, the only difference is in the 4th column which represents the number of reviews. The more reviews should mean the more recent the entry, so we will keep the most recent entry by keeping the one with the highest review count. We will do this by deleting the duplicates for each app that don't have the highest review count. 

In [41]:
reviews_max = {}

for app in google:
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
        
print("Expected Length:",len(google)-1181)
print("Actual Length:",len(reviews_max))
        

Expected Length: 9659
Actual Length: 9659


This shows that our new dictionary length of app names and number of reviews matches the expected length.

In [42]:
google_clean = []
already_added = []

for app in google:
    name = app[0]
    n_reviews = float(app[3])
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        google_clean.append(app)
        already_added.append(name)

If the number of reviews matches the max number of reviews for that corresponding app, then it is added to the google_clean list. The already_added list is to account for any apps that have multiple entries with the same number of reviews.

In [43]:
explore_data(google_clean,0,4,True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


Number of rows: 9659
Number of columns: 13


This shows that in our new google_clean list we have the expected number of rows/apps.

# Removing Non-English Apps

All characters have a corresponding number to them, in the ASCII system english text character numbers range from 0 to 127. We will check the names of the apps and remove the ones that are not within this range.

Below we will write a function that determines whether a string contains all english characters or not.

In [44]:
def is_english(string):
    for character in string:
        if ord(character) > 127:
            return False
    return True

print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
False
False
False


Clearly there are some cicumstances where an english app has characters that are above the 127 ASCII range. To minimize the impact of data loss, we'll only remove an app if its name has more than three characters with corresponding numbers falling outside the ASCII range. This means all English apps with up to three emoji or other special characters will still be labeled as English. Our filter function is still not perfect, but it should be fairly effective.

In [45]:
def is_english(string):
    non_english = 0
    
    for character in string:
        if ord(character) > 127:
            non_english += 1
            
    if non_english > 3:
        return False
    
    return True

print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
False
True
True


This function seems to be sufficient although not perfect, below we will filter out the non-english apps using this function.

In [46]:
english_google = []
english_appstore = []

for app in google_clean:
    if is_english(app[0]):
        english_google.append(app)
        
for app in appstore:
    if is_english(app[1]):
        english_appstore.append(app)
        
explore_data(english_google,0,3,True)
print('\n')
explore_data(english_appstore,0,3,True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 

We are left with 9614 apps from Google Play and 6183 apps from The Appstore

# Isolating Free Apps

In [47]:
free_appstore = []
free_google = []

for app in english_appstore:
    if app[4] == '0.0':
        free_appstore.append(app)
        
for app in english_google:
    if app[7] == '0':
        free_google.append(app)
        
print(len(free_appstore))
print(len(free_google))

3222
8864


After isolating for free apps, we are left with 3222 apps in the Appstore and 8864 apps in the Google Play store.

# Most Common Apps by Genre

Our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

We'll build a frequency table for the prime_genre column of the App Store data set, and the Genres and Category columns of the Google Play data set. This will help us figure out the most common genres. 

Below we'll build two functions we can use to analyze the frequency tables:

1. One function to generate frequency tables that show percentages
2. Another function that we can use to display the percentages in a descending order

In [48]:
def freq_table(dataset,index):
    freq = {}
    total = 0
    for row in dataset:
        total += 1
        value = row[index]
        if value in freq:
            freq[value] += 1
        else:
            freq[value] = 1
            
    percentages = {}
    for number in freq:
        percent = (freq[number] / total) * 100
        percentages[number] = percent
    
    return percentages

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [49]:
display_table(free_appstore,-5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


In [50]:
display_table(free_google,1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [51]:
display_table(free_google,-4)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

From the google play dataset we will only use the Categories column moving forward, the genres column is too granular.

For The App Store the most common Genre is Games, with runner up Entertainment. For Google Play the most common is Family with the runner up being Games. It seems The App Store is more for entertainment purposes and Google Play is a bit more practical focused. 

# Most Popular Apps by Genre on the App Store

We'll take the total number of user ratings, since there is no column showing number of installs for the App Store.

Below, we calculate the average number of user ratings per app genre on the App Store:

In [52]:
prime_genre_freq = freq_table(free_appstore,-5)
table_display = []

for genre in prime_genre_freq:
    total = 0   #to store number of ratings per genre
    len_genre = 0   #to store number of apps in each genre
   
    for app in free_appstore:
        genre_app = app[-5]
        if genre_app == genre:
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total/len_genre
    
    key_val_as_tuple = (avg_n_ratings, genre)
    table_display.append(key_val_as_tuple)
        
table_sorted = sorted(table_display, reverse = True)
for entry in table_sorted:
    print(entry[1], ':', entry[0])

Navigation : 86090.33333333333
Reference : 74942.11111111111
Social Networking : 71548.34905660378
Music : 57326.530303030304
Weather : 52279.892857142855
Book : 39758.5
Food & Drink : 33333.92307692308
Finance : 31467.944444444445
Photo & Video : 28441.54375
Travel : 28243.8
Shopping : 26919.690476190477
Health & Fitness : 23298.015384615384
Sports : 23008.898550724636
Games : 22788.6696905016
News : 21248.023255813954
Productivity : 21028.410714285714
Utilities : 18684.456790123455
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Business : 7491.117647058823
Education : 7003.983050847458
Catalogs : 4004.0
Medical : 612.0


Below we will print out the apps from the top 5 categories along with their number of ratings.

In [53]:
for app in free_appstore:
    if app[-5] == 'Navigation':
        print(app[-5], ':', app[1], ':', app[5])

print('\n')
for app in free_appstore:
    if app[-5] == 'Reference':
        print(app[-5], ':', app[1], ':', app[5])

print('\n')
for app in free_appstore:
    if app[-5] == 'Social Networking':
        print(app[-5], ':', app[1], ':', app[5])

print('\n')
for app in free_appstore:
    if app[-5] == 'Music':
        print(app[-5], ':', app[1], ':', app[5])

print('\n')
for app in free_appstore:
    if app[-5] == 'Weather':
        print(app[-5], ':', app[1], ':', app[5])

Navigation : Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Navigation : Google Maps - Navigation & Transit : 154911
Navigation : Geocaching® : 12811
Navigation : CoPilot GPS – Car Navigation & Offline Maps : 3582
Navigation : ImmobilienScout24: Real Estate Search in Germany : 187
Navigation : Railway Route Search : 5


Reference : Bible : 985920
Reference : Dictionary.com Dictionary & Thesaurus : 200047
Reference : Dictionary.com Dictionary & Thesaurus for iPad : 54175
Reference : Google Translate : 26786
Reference : Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
Reference : New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Reference : Merriam-Webster Dictionary : 16849
Reference : Night Sky : 12122
Reference : City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
Reference : LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
Reference : GUNS MODS for Minecra

It looks as if in the really popular categories there are a few "app giants" such as facebook, or spotify that have such an extremely large following they throw off the average. These popular cateories may not be optimal for new apps entering their market.

In [54]:
for app in free_appstore:
    if app[-5] == 'Health & Fitness':
        print(app[-5], ':', app[1], ':', app[5])

Health & Fitness : Calorie Counter & Diet Tracker by MyFitnessPal : 507706
Health & Fitness : Lose It! – Weight Loss Program and Calorie Counter : 373835
Health & Fitness : Weight Watchers : 136833
Health & Fitness : Sleep Cycle alarm clock : 104539
Health & Fitness : Fitbit : 90496
Health & Fitness : Period Tracker Lite : 53620
Health & Fitness : Nike+ Training Club - Workouts & Fitness Plans : 33969
Health & Fitness : Plant Nanny - Water Reminder with Cute Plants : 27421
Health & Fitness : Sworkit - Custom Workouts for Exercise & Fitness : 16819
Health & Fitness : Clue Period Tracker: Period & Ovulation Tracker : 13436
Health & Fitness : Headspace : 12819
Health & Fitness : Fooducate - Lose Weight, Eat Healthy,Get Motivated : 11875
Health & Fitness : Runtastic Running, Jogging and Walking Tracker : 10298
Health & Fitness : WebMD for iPad : 9142
Health & Fitness : 8fit - Workouts, meal plans and personal trainer : 8730
Health & Fitness : Garmin Connect™ Mobile : 8341
Health & Fitness 

All the apps in Health & Fitness seem to be more around the average. Running, calorie tracking, and weight tracking apps seem to be quite popular, but these 3 factors seem to be seperated in different apps. Perhaps an app that entails all of these aspects would do quite well.

# Most Popular Apps by Genre on Google Play

In [55]:
display_table(free_google, 5) # the Installs columns

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


We don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. But we don't need very precise data for our purposes.

We'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

We'll need to convert each install number to float — this means that we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error.

In [56]:
category_freq = freq_table(free_google,1)
table_display = []

for category in category_freq:
    total = 0   #to store number of ratings per genre
    len_category = 0   #to store number of apps in each genre
   
    for app in free_google:
        category_app = app[1]
        if category_app == category:
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
            
    avg_n_installs = total/len_category
    
    key_val_as_tuple = (avg_n_installs, category)
    table_display.append(key_val_as_tuple)
        
table_sorted = sorted(table_display, reverse = True)
for entry in table_sorted:
    print(entry[1], ':', entry[0])

COMMUNICATION : 38456119.167247385
VIDEO_PLAYERS : 24727872.452830188
SOCIAL : 23253652.127118643
PHOTOGRAPHY : 17840110.40229885
PRODUCTIVITY : 16787331.344927534
GAME : 15588015.603248259
TRAVEL_AND_LOCAL : 13984077.710144928
ENTERTAINMENT : 11640705.88235294
TOOLS : 10801391.298666667
NEWS_AND_MAGAZINES : 9549178.467741935
BOOKS_AND_REFERENCE : 8767811.894736841
SHOPPING : 7036877.311557789
PERSONALIZATION : 5201482.6122448975
WEATHER : 5074486.197183099
HEALTH_AND_FITNESS : 4188821.9853479853
MAPS_AND_NAVIGATION : 4056941.7741935486
FAMILY : 3695641.8198090694
SPORTS : 3638640.1428571427
ART_AND_DESIGN : 1986335.0877192982
FOOD_AND_DRINK : 1924897.7363636363
EDUCATION : 1833495.145631068
BUSINESS : 1712290.1474201474
LIFESTYLE : 1437816.2687861272
FINANCE : 1387692.475609756
HOUSE_AND_HOME : 1331540.5616438356
DATING : 854028.8303030303
COMICS : 817657.2727272727
AUTO_AND_VEHICLES : 647317.8170731707
LIBRARIES_AND_DEMO : 638503.734939759
PARENTING : 542603.6206896552
BEAUTY : 51315

The most popular categories probably have skewed averages due to extremely dominant apps, like the App Store.
So we will look at a less popular category like Sports.

In [58]:
for app in free_google:
    if app[1] == 'SPORTS':
        print(app[0], ':', app[5])

Free Sports TV : 100,000+
LiveScore: Live Sport Updates : 10,000,000+
Onefootball - Soccer Scores : 10,000,000+
Cristiano Ronaldo Wallpaper : 500,000+
Futbol24 : 1,000,000+
kicker football news : 5,000,000+
Football Live Scores : 5,000,000+
Pro 2018 - Series A and B : 5,000,000+
BeSoccer - Soccer Live Score : 10,000,000+
Sport.pl LIVE : 1,000,000+
The Team - Live Sport: football, tennis, rugby .. : 5,000,000+
MARCA - Sports Leader Diary : 5,000,000+
Sahadan Live Scores : 5,000,000+
Foot Mercato: transfers, results, news, live : 1,000,000+
All Football - Latest News & Videos : 10,000,000+
Eurosport : 10,000,000+
beIN SPORTS TR : 10,000,000+
All Football GO- Live Score, Games : 100,000+
Top Mercato: football news : 1,000,000+
GollerCepte Live Score : 1,000,000+
Championat : 1,000,000+
Hovercraft Racer : 100,000+
World Cup 2018 : 500,000+
GollerCepte 1903 : 500,000+
League18 : 500,000+
Speed Boat Racing : 1,000,000+
La Liga - Spanish Soccer League Official : 10,000,000+
Lionel Messi Wallp

It looks like there is room for an app that displays live data about all the different sports, while having fantasy access.